In [2]:
import os
import numpy as np
from sklearn.cluster import KMeans
from mayavi import mlab
from phathom.phenotype.mesh import randomly_sample

In [3]:
working_dir = '/media/jswaney/SSD EVO 860/organoid_phenotyping/20181206_eF9_A34_1'

In [4]:
centers = np.load(os.path.join(working_dir, 'centers.npy'))
centers.shape

(2421822, 3)

In [14]:
sox2_labels_path = 'sox2_labels.npy'
tbr1_labels_path = 'tbr1_labels.npy'

sox2_labels = np.load(os.path.join(working_dir, sox2_labels_path))
tbr1_labels = np.load(os.path.join(working_dir, tbr1_labels_path))

In [5]:
voxel_size = np.array((2.052, 1.082, 1.082))

centers_um = centers * voxel_size
centers_um.shape

(2421822, 3)

# KMeans on nuclei centroids

In [6]:
n_clusters = 3

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(centers_um)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_
labels

array([2, 2, 2, ..., 1, 1, 1], dtype=int32)

In [7]:
# Extract indices for each cluster
indices = [np.where(labels == c)[0] for c in range(n_clusters)]
len(indices[0]) 

803309

In [10]:
n = 10000

samples = np.asarray([randomly_sample(n, idx)[0] for idx in indices])
samples.shape

(3, 10000)

In [13]:
# Plot the random samples
scale_factor = 25

np.random.seed(1)
for c in range(n_clusters):
    idx = samples[c]
    coords = centers_um[idx]
    mlab.points3d(coords[:, 0], 
                  coords[:, 1], 
                  coords[:, 2], 
                  scale_factor=scale_factor, 
                  color=tuple(np.random.random(3)))
mlab.show()

In [32]:
tuple(np.random.random(3))

(0.037659244861324326, 0.0010135773904080825, 0.7403490197774311)

# 3D render of cell types in whole org

In [15]:
dn_labels = ~np.logical_or(sox2_labels, tbr1_labels)
dn_labels.shape

(2421822,)

In [17]:
centers_sox2 = centers_um[np.where(sox2_labels)]
centers_tbr1 = centers_um[np.where(tbr1_labels)]
centers_dn = centers_um[np.where(dn_labels)]
centers_sox2.shape, centers_tbr1.shape, centers_dn.shape

((622435, 3), (131008, 3), (1693543, 3))

In [47]:
(samples_sox2,) = randomly_sample(20000, centers_sox2)
(samples_tbr1,) = randomly_sample(10000, centers_tbr1)
(samples_dn,) = randomly_sample(20000, centers_dn)

In [ ]:
# Plot the random samples
scale_factor = 15

mlab.points3d(samples_sox2[:, 0], 
              samples_sox2[:, 1], 
              samples_sox2[:, 2], 
              scale_factor=scale_factor, 
              color=(1, 0, 0))

mlab.points3d(samples_tbr1[:, 0], 
              samples_tbr1[:, 1], 
              samples_tbr1[:, 2], 
              scale_factor=scale_factor, 
              color=(0, 0.95, 0))

mlab.points3d(samples_dn[:, 0], 
              samples_dn[:, 1], 
              samples_dn[:, 2], 
              scale_factor=scale_factor, 
              color=(0, 0, 0.9))
    
mlab.show()